In [ ]:
import trimesh
import open3d as o3d
import trimesh.exchange.stl

import numpy as np
import matplotlib.pyplot as plt

from common_functions import *

# Modify grain geometry from tomography output

In [ ]:
isolated_grains = trimesh.load_mesh('grain-geometries-CAD/box1_isolatedgrains-packed3.stl') 
# # scaling seems to be in mm, no need to adjust manually. otherwise, use:
# isolated_grains.apply_scale(1e-3)

In [ ]:
isolated_grains.show()

In [ ]:
isolated_grains.is_watertight

In [ ]:
# Convert Trimesh → Open3D
mesh_o3d = o3d.geometry.TriangleMesh(
    vertices=o3d.utility.Vector3dVector(isolated_grains.vertices),
    triangles=o3d.utility.Vector3iVector(isolated_grains.faces)
)

# Compute normals (optional but useful for remeshing)
mesh_o3d.compute_vertex_normals()

# Apply Loop subdivision
mesh_remesh = mesh_o3d.subdivide_loop(number_of_iterations=2)

# Convert Open3D → Trimesh
vertices = np.asarray(mesh_remesh.vertices)
faces = np.asarray(mesh_remesh.triangles)
isolated_grains_remeshed = trimesh.Trimesh(vertices=vertices, faces=faces, process=False)

# Visualize with Trimesh
isolated_grains_remeshed.show()

In [ ]:
# The matrix you defined:
flip_transform = np.array([[1, 0, 0, 0],
                           [0, 1, 0, 0],
                           [0, 0, -1, 0], 
                           [0, 0, 0, 1]]) # Added the necessary 4th row for homogeneous coordinates

# CORRECT: Use apply_transform() for matrix operations
isolated_grains_remeshed.apply_transform(flip_transform)

In [ ]:
isolated_grains_remeshed.show()

In [ ]:
centroid = isolated_grains_remeshed.centroid
# Translate the mesh so that the centroid is at (0, 0, 0)
isolated_grains_remeshed.apply_translation(-centroid)

# Get bounding box corners
min_corner, max_corner = isolated_grains_remeshed.bounds  # shape (2, 3)

# Compute size along each axis (X, Y, Z)
size = max_corner - min_corner

print(f"Bounding box min corner: {min_corner}")
print(f"Bounding box max corner: {max_corner}")
print(f"Size (X x Y x Z): {size}")

In [ ]:
isolated_grains_remeshed.apply_translation((0,0,-0.015))

In [ ]:
# Areas of all triangles (in the mesh surface)
triangle_areas = isolated_grains_remeshed.area_faces  # shape: (num_faces,)

# Average area
mean_area = triangle_areas.mean()
print(f"Mean triangle area: {mean_area*1000*1000:.8f} nm")
# should translate to an area of around 1 micron... in which case we should not go above a map spacing of 2 microns

plt.hist(triangle_areas*1000*1000, bins=100)
plt.show()

In [ ]:
## determine appropiate grid spacing for Electric Field calculations

# Create mesh grid for exact sampling
WorldX, WorldY, WorldZ = 117.00044-10, 120.77438, 100.29045+15
print(117.00044-10, 120.77438, 100.29045)
stepsize = 10

x_array = np.arange(-WorldX/2, WorldX/2, stepsize)/1000
y_array = np.arange(-WorldY/2, WorldY/2, stepsize)/1000
z_array = np.arange(-WorldZ/2, WorldZ/2, stepsize)/1000

# Create 3D mesh grid
X, Y, Z = np.meshgrid(x_array, y_array, z_array, indexing='ij')

# Flatten the mesh grid to create sampling points
sampling_points = np.column_stack([X.ravel(), Y.ravel(), Z.ravel()])
print(len(sampling_points))
photoelectron_stopping_sites = trimesh.points.PointCloud(sampling_points, colors=[0, 0, 255, 255])

scene = plot_trimesh_edges_only(isolated_grains_remeshed, edge_color=[0, 0, 0, 128])
scene.add_geometry([photoelectron_stopping_sites])
scene.show()

In [ ]:
# Get ASCII STL string
ascii_stl_str = trimesh.exchange.stl.export_stl_ascii(mesh=isolated_grains_remeshed)

# Write it to a file
with open("../sphere-charging/geometry/isolated_grains_interpolated.stl", "w") as f:
    f.write(ascii_stl_str)

In [ ]:
isolated_grains_remeshed.fill_holes()

In [ ]:
isolated_grains_remeshed.is_watertight

In [ ]:
# --- Comprehensive Repair Steps (Order is crucial) ---

# 2a. Aggressively merge coincident vertices. This is crucial for fixing non-manifold
# geometry or tiny gaps that prevent proper boundary finding.
# The default tolerance is usually effective.
isolated_grains_remeshed.merge_vertices()
print("    -> Merged vertices to clean up geometry.")

# 2b. Fix mesh winding to ensure normals are consistent (important after the Z-flip)
trimesh.repair.fix_winding(isolated_grains_remeshed)
print("    -> Fixed mesh winding.")

# 2c. Force a full reprocessing and fix inversion
isolated_grains_remeshed.process(validate=True) 
trimesh.repair.fix_inversion(isolated_grains_remeshed)

# Final process after geometric repairs
isolated_grains_remeshed.process() 

# Check status again
is_watertight_after_repair = isolated_grains_remeshed.is_watertight

if is_watertight_after_repair:
    print("Repair successful! The mesh is now watertight.")
else:
    # Warning adjusted as fill_holes was intentionally skipped
    print("Warning: Repair failed. Mesh remains non-watertight (requires hole filling).")
    

In [ ]:
isolated_grains_remeshed.is_watertight

In [ ]:
broken = trimesh.repair.broken_faces(isolated_grains_remeshed, color=None)